## Обзор данных

Составим первое представление о данных Яндекс.Музыки.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('yandex_music_project.csv') # чтение файла с данными и сохранение в df

In [3]:
display(df.head(10)) # получение первых 10 строк таблицы df

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Найдите ещё одну проблему в названии колонок и опишите её в этом пункте.



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


In [5]:
df.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63848,57876,63881,65079,65079,65079
unique,41748,47245,43605,289,2,20392,3
top,A8AE9169,Intro,Sasha,pop,Moscow,08:14:07,Friday
freq,76,34,6,8850,45360,14,23149


В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных
Исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

### Стиль заголовков
Выведем на экран названия столбцов:

In [6]:
df.columns # перечень названий столбцов таблицы df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведем названия в соответствие с хорошим стилем:
* несколько слов в названии запишем в «змеином_регистре»,
* все символы сделаем строчными,
* устраним пробелы.

Для этого переименуем колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [7]:
df = df.rename(columns = {'  userID':'user_id',
                          'Track':'track',
                          '  City  ':'city',
                          'Day':'day'}) # переименование столбцов

Проверим результат. Для этого ещё раз выведем на экран названия столбцов:

In [8]:
display(df.columns) # проверка результатов - перечень названий столбцов

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений
Сначала посчитаем, сколько в таблице пропущенных значений. Для этого достаточно двух методов `pandas`:

In [9]:
display(df.isna().sum()) # подсчёт пропусков

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для нашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

In [10]:
columns_to_replace = ['track', 'artist', 'genre']    # объявите список columns_to_replace с названиями столбцов track, artist, genre
df[columns_to_replace] = df[columns_to_replace].fillna('unknown') #замена пропущенных значений на 'unknown'

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [11]:
display(df.isna().sum()) # подсчёт пропусков

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты
Посчитаем явные дубликаты, удалим их:

In [12]:
display(df.duplicated().sum()) # подсчёт явных дубликатов

3826

In [13]:
df=df.drop_duplicates().reset_index(drop=True)  # удаление явных дубликатов (с удалением старых индексов и формированием новых)

In [14]:
display(df.duplicated().sum()) # проверка на отсутствие дубликатов

0

Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке:

In [38]:
genre_sorted = sorted(df['genre'].unique())
print(np.transpose(genre_sorted) )

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

Просмотрим список и найдем неявные дубликаты названия `hiphop`. Это могут быть названия с ошибками или альтернативные названия того же жанра.

Мы увидим следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, напишем функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция должна исправить колонку `genre` в таблице `df`: заменить каждое значение из списка `wrong_genres` на значение из `correct_genre`.

In [16]:
def replace_wrong_genres(wrong_genres,correct_genre): # Функция для замены неявных дубликатов
    for wrong_genre in wrong_genres:  # перебираем неправильные имена
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre) # и для каждого неправильного имени вызываем метод replace()

Вызовем `replace_wrong_genres()` и передадим ей такие аргументы, чтобы она устранила неявные дубликаты: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [17]:
duplicates = ['hip', 'hop', 'hip-hop']
correct_name = 'hiphop'  

df['genre'] = df['genre'].replace(duplicates, correct_name) # Устранение неявных дубликатов

Проверим, что заменили неправильные названия:

*   hip
*   hop
*   hip-hop

Выведем отсортированный список уникальных значений столбца `genre`:

In [18]:
genre_sorted = sorted(df['genre'].unique())
for item in genre_sorted:
    print(item, end=',  ')         # Проверка на неявные дубликаты

acid,  acoustic,  action,  adult,  africa,  afrikaans,  alternative,  alternativepunk,  ambient,  americana,  animated,  anime,  arabesk,  arabic,  arena,  argentinetango,  art,  audiobook,  author,  avantgarde,  axé,  baile,  balkan,  beats,  bigroom,  black,  bluegrass,  blues,  bollywood,  bossa,  brazilian,  breakbeat,  breaks,  broadway,  cantautori,  cantopop,  canzone,  caribbean,  caucasian,  celtic,  chamber,  chanson,  children,  chill,  chinese,  choral,  christian,  christmas,  classical,  classicmetal,  club,  colombian,  comedy,  conjazz,  contemporary,  country,  cuban,  dance,  dancehall,  dancepop,  dark,  death,  deep,  deutschrock,  deutschspr,  dirty,  disco,  dnb,  documentary,  downbeat,  downtempo,  drum,  dub,  dubstep,  eastern,  easy,  electronic,  electropop,  emo,  entehno,  epicmetal,  estrada,  ethnic,  eurofolk,  european,  experimental,  extrememetal,  fado,  fairytail,  film,  fitness,  flamenco,  folk,  folklore,  folkmetal,  folkrock,  folktronica,  f

Также, при просмотре уникальных жанров музыки, я обнаружила жанр 'ïîï' - где видимо случились проблемы с кодировкой, разобравшись с кодировкой, это оказался неявный дубликат жанра 'pop'. Поэтому, запустим функцию по замене дубликатов еще раз, с другими входными данными (и проверим потом еще раз через вывод, что данный дубликат устранен):

In [19]:
duplicates = ['ïîï']
name = 'pop'
replace_wrong_genres(duplicates, name) # Устранение неявных дубликатов
genre_sorted = sorted(df['genre'].unique())
for item in genre_sorted:
    print(item, end=',  ')

acid,  acoustic,  action,  adult,  africa,  afrikaans,  alternative,  alternativepunk,  ambient,  americana,  animated,  anime,  arabesk,  arabic,  arena,  argentinetango,  art,  audiobook,  author,  avantgarde,  axé,  baile,  balkan,  beats,  bigroom,  black,  bluegrass,  blues,  bollywood,  bossa,  brazilian,  breakbeat,  breaks,  broadway,  cantautori,  cantopop,  canzone,  caribbean,  caucasian,  celtic,  chamber,  chanson,  children,  chill,  chinese,  choral,  christian,  christmas,  classical,  classicmetal,  club,  colombian,  comedy,  conjazz,  contemporary,  country,  cuban,  dance,  dancehall,  dancepop,  dark,  death,  deep,  deutschrock,  deutschspr,  dirty,  disco,  dnb,  documentary,  downbeat,  downtempo,  drum,  dub,  dubstep,  eastern,  easy,  electronic,  electropop,  emo,  entehno,  epicmetal,  estrada,  ethnic,  eurofolk,  european,  experimental,  extrememetal,  fado,  fairytail,  film,  fitness,  flamenco,  folk,  folklore,  folkmetal,  folkrock,  folktronica,  f

Мы обнаружили еще несколько похожих жанров, например 'africa' и 'afrikaans', но я не стала их исправлять, так как они далеки от поп музыки. Если `pop`  в неправильной кодировке был, а обычно он относится к популярным жанрам, то такой дубликат исказит анализ. с другой стороны,  'afrikaans' вряд-ли относится к популярным жанрам, и исказит анализ меньше.

В итоге, предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Мы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения мы заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Сгруппируем данные по городу и посчитаем прослушивания в каждой группе:

In [20]:
display(df.groupby('city')['city'].count()) # Подсчёт прослушиваний в каждом городе

city
Moscow              42741
Saint-Petersburg    18512
Name: city, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитайте прослушивания в понедельник, среду и пятницу. Учтем, что в данных есть информация только о прослушиваниях только за эти дни.


In [21]:
display(df.groupby('day')['day'].count()) # Подсчёт прослушиваний в каждый из трёх дней

day
Friday       21840
Monday       21354
Wednesday    18059
Name: day, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраним в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Затем посчитаем значения в столбце `user_id` получившейся таблицы, результат сохраним в новую переменную и вернем ее из функции.

In [22]:
def number_tracks(day, city): 
    '''функция, которая сначала создает отфильтрованный
    по параметрам day (день недели), city (город) DataFrame, 
    а потом подсчитывает количество строк, удовлетворяющих фильтру '''    
    track_list = df[df['day'] == day]  
    track_list = track_list[track_list['city'] == city] 
    track_list_count = track_list['user_id'].count()  
    return track_list_count

Посчитаем с помощью функции сколько в какой день было прослушиваний:

In [23]:
for weekday, city in zip(['Monday', 'Wednesday', 'Friday']*2, sorted(['Moscow', 'Saint-Petersburg']*3)):
    print(f'Количество прослушиваний в городе {city} в {weekday} равно {number_tracks(weekday, city)}')

Количество прослушиваний в городе Moscow в Monday равно 15740
Количество прослушиваний в городе Moscow в Wednesday равно 11056
Количество прослушиваний в городе Moscow в Friday равно 15945
Количество прослушиваний в городе Saint-Petersburg в Monday равно 5614
Количество прослушиваний в городе Saint-Petersburg в Wednesday равно 7003
Количество прослушиваний в городе Saint-Petersburg в Friday равно 5895


Создадим c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [24]:
name_columns = ['city', 'monday', 'wednesday', 'friday']
track_city_counts = []
for item_city in ['Moscow', 'Saint-Petersburg']:
    row = []
    row.append(item_city)
    for item_day in ['Monday', 'Wednesday', 'Friday']:
        row.append(number_tracks(item_day, item_city))
    track_city_counts.append(row)
display(pd.DataFrame(data = track_city_counts ,columns = name_columns) )   # Таблица с результатами

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [25]:
moscow_general = df[df['city'] == 'Moscow']# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
display(moscow_general)

,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday
...,...,...,...,...,...,...,...
61247,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Moscow,21:07:12,Monday
61248,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
61250,C5E3A0D5,Jalopiina,unknown,industrial,Moscow,20:09:26,Friday
61251,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday


In [26]:
spb_general = df[df['city'] == 'Saint-Petersburg']# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
display(spb_general)

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday
...,...,...,...,...,...,...,...
61239,D94F810B,Theme from the Walking Dead,Proyecto Halloween,film,Saint-Petersburg,21:14:40,Monday
61240,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Saint-Petersburg,21:06:50,Monday
61241,29E04611,Bre Petrunko,Perunika Trio,world,Saint-Petersburg,13:56:00,Monday
61242,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Saint-Petersburg,09:22:13,Monday


Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [27]:
def genre_weekday(table, day, time1, time2):
    '''Функция возвращает объект Series на основе таблицы (table) о 10 самых популярных жанрах 
    в указанный день (day) в заданное время (от time1 до time2 не включая границ)'''
    genre_df = table[table['day'] == day]
    genre_df = genre_df [genre_df['time'] > time1]
    genre_df = genre_df [genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['user_id'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    return genre_df_sorted.head(10)

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [28]:
# вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
# объекты, хранящие время, являются строками и сравниваются как строки
# пример вызова: genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64

In [29]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64

In [30]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: user_id, dtype: int64

In [31]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: user_id, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [32]:
moscow_genres = moscow_general.groupby('genre')['user_id'].count().sort_values(ascending=False)

In [33]:
# просмотр первых 10 строк moscow_genres
moscow_genres.to_frame().head(10)

,user_id
genre,
pop,5893
dance,4435
rock,3965
electronic,3786
hiphop,2096
classical,1616
world,1432
alternative,1379
ruspop,1372


Теперь повторим то же и для Петербурга, сгруппируем таблицу `spb_general` по жанру, посчитаем прослушивания треков каждого жанра. Результат отсортируем в порядке убывания и сохраним в таблице `spb_genres`:


In [39]:
spb_genres = spb_general.groupby('genre')['user_id'].count().sort_values(ascending=False)
display(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: user_id, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Выводы
Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.